In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, RandomAffine, ToTensor

# Not included by default in Colab
!pip install torchinfo
from torchinfo import summary

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

# For presentations purposes only (not needed in Colab)
#plt.style.use('notebook.mplstyle')
# Keeps the kernel from dying in notebooks on Windows machines (not needed in Colab)
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Download the MNIST dataset

In [ ]:
# Augmentations
#No agumentations (default)
augmentations = Compose([ToTensor()])
# Add random affine transformations
#augmentations = Compose([RandomAffine(degrees=(-30, 30), translate=(0.1, 0.1), scale=(0.85, 1.1)), ToTensor()])

# Download the MNIST dataset
# Use datasets.MNIST for the classic numeric dataset
# or datasets.FashionMNIST for one with clothes

# Download training data.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=augmentations,
)

# Download test data.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

### Show an example image and illustrate that the data is pixel values

In [ ]:
# Extract one example image
example_img = training_data.data[1]
# Use a pandas dataframe to show the whole array with pixel values colorcoded
df = pd.DataFrame(example_img)
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

### Attach the data to PyTorch DataLoaders
The dataloader will provide batches of a predefined batch size during training.

In [ ]:
# The number of images to process in one batch 
# before making a model parameter update.
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print('The training set contains {:d} images'.format(len(train_dataloader.dataset)))
print('The test set contains {:d} images'.format(len(test_dataloader.dataset)))

# Examplify the shape of X ad y in one batch
print('\nOne batch contains:')
for X, y in test_dataloader:
    print(f"  X [N, C, H, W]: {X.shape}")
    print(f"  y [N]: {y.shape} {y.dtype}")
    break

### Define various types of networks
We will define and compare three different networks:
1. Simple logistic regression (a fully connected network without hidden layers).
2. A fully connected network with two hidden layers.
3. A convolutional neural network with five hidden layers.

In [ ]:
# Define a logistic regression model using PyTorch
class LogReg(nn.Module):
    def __init__(self):
        super(LogReg, self).__init__()
        self.flatten = nn.Flatten()  
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 10),  # 28*28 --> 10
        )

    def forward(self, x):
        x = self.flatten(x)  # 28x28 --> 28*28
        logits = self.linear_stack(x)
        return logits

# Define a fully connected neural network
# with 2 hidden layers
# and dropout for regularization
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),  # 28*28 --> 512
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 512),    # 512 --> 512
            nn.ReLU(),
            nn.Linear(512, 10)      # 512 --> 10
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Define a deep convolutional neural network
# and dropout for regularization
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # nn.Linear(in_features, out_features)
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 128, 3, 1, 1),   # 1x28x28 ==> 64x28x28
            nn.ReLU(),
            nn.Conv2d(128, 64, 3, 1, 1),  # 64x28x28 ==> 64x28x28
            nn.ReLU(),
            nn.MaxPool2d(2),             # 64x28x28 ==> 64x14x14
            nn.Conv2d(64, 64, 3, 1, 1),  # 64x14x14 ==> 64x14x14
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),  # 64x14x14 ==> 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(2),             # 64x14x14 ==> 64x7x7
            nn.Flatten(),                # 64x7x7 ==> 64*7*7
            nn.Dropout(0.5),
            nn.Linear(3136, 256),        # 64*7*7 ==> 256
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
      
        logits = self.linear_relu_stack(x)
        return logits

# ---------------------------------------------------- #
# Modern networks are usuassy build by defining smaller blocks that 
# have empirically been shown to do useful things.
# Here is one example where we define a convolutional block with
# residual connections from which residual networks can be built.

class Residual(nn.Module):
    """The Residual block of ResNet."""
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = nn.functional.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return nn.functional.relu(Y)

# Define a deeper convolutional neural network
# from residual blocks.
class ResNet(nn.Module):
    """Buil a residual type net from multiple residual blocks"""
    def __init__(self):
        super().__init__()
        self.res_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),    # 1x28x28 --> 32x28x28
            nn.ReLU(),
            Residual(32, 32),                              # 32x28x28 --> 32x28x28
            Residual(32, 64, use_1x1conv=True, strides=2), # 32x28x28 --> 64x14x14
            Residual(64, 64),                              # 64x14x14 --> 64x14x14
            Residual(64, 64, use_1x1conv=True, strides=2), # 64x14x14 --> 64x7x7
            nn.Flatten(),            # 64x7x7 --> 64*7*7
            nn.Dropout(0.5),
            nn.Linear(7*7*64, 256),  # 64*7*7 --> 256
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)       # 256 --> 10
        )

    def forward(self, X):
        logits = self.res_stack(X)
        return logits

### Select the network we want to use and move it to the GPU

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Select the model that we would like to use
#model = LogReg().to(device)
#model = MLP().to(device)
model = ConvNet().to(device)
#model = ResNet().to(device)

# Show summary over the model and the number of parameters that it has
print(summary(model, input_size=(batch_size, 1, 28, 28)))

### Define training and testing functions

In [ ]:
def train(dataloader, model, loss_fn, optimizer, scheduler):
    
    # Check how many images the training set contains
    size = len(dataloader.dataset)
    # Certain layers (e.g. dropout) work differently during
    # training and testing ==> use for training
    model.train()
    
    # Loop over all batches
    for batch, (X, y) in enumerate(dataloader):
        
        # Move the data to the GPU
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Write out progress for 100th batch
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print(f"lr: {scheduler.get_last_lr()[0]:1.6f}")

            
def test(dataloader, model, loss_fn):
    
    # Check how many images the training set contains
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # Certain layers (e.g. dropout) work differently during
    # training and testing ==> use for testing (evaluation)
    model.eval()
    
    test_loss, correct = 0, 0
    # Speed up computations by not computing gradients
    with torch.no_grad():
        
        # Loop over all batches in the test set
        for X, y in dataloader:
            
            # Move the data to the GPU
            X, y = X.to(device), y.to(device)
            
            # Compute prediction error
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            # count how many images that were classified correctly
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    # Compute and print the average loss and accuracy
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Train the selected network
Observe that all defined networks above have a final linear layer. The CrossEntropy loss function takes care of the softmax activation functions, as it is not neccessary to compute probabilities for selecting the most likely output.

In [ ]:
n_epochs = 15

# Use cross entropy loss as this is a classification task
loss_fn = nn.CrossEntropyLoss()

# Use Adam as our optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
# It is common to observe that performance can be improved by annealing the
# learnign rate over time. The easiest way to achieve this is by using a 
# scheduler that adjusts it within a predefined interval.
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, 
                                                steps_per_epoch=len(train_dataloader), epochs=n_epochs, 
                                                div_factor=10, final_div_factor=100)

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, scheduler)
    test(test_dataloader, model, loss_fn)

### Make predictions for the whole training set

In [ ]:
# Make one run over the whole training set 
# and store away predictions and iamges in 
# separate numpy arrays.

X = []
y = []
y_hat = []
with torch.no_grad():
    
    # Certain layers (e.g. dropout) work differently during
    # training and testing ==> use for testing (evaluation)
    model.eval()
    
    # Loop over the test set
    for X_tmp, y_tmp in test_dataloader:
        
        # Move data to the GPU
        X_tmp, y_tmp = X_tmp.to(device), y_tmp.to(device)
        # Predict the label
        pred = model(X_tmp)
        
        # Store all data
        X.append(X_tmp.cpu().detach().numpy())
        y.append(y_tmp.cpu().detach().numpy())
        y_hat.append(pred.argmax(1).cpu().detach().numpy())
        
y = np.hstack(y)
y_hat = np.hstack(y_hat)
X = np.vstack(X)

### Visualize example images that were incorrectly labeled

In [ ]:
# Find incorrectly predicted images
incorrect_idx = np.where(y != y_hat)[0]

# Create a figure window
fig = plt.figure(figsize=[17, 14])

# Loop over incorrectly classified images and plot examples
for i in range(25):
    # Create a subplot
    ax = fig.add_subplot(5, 5, i+1)
    # Select an incorreclty classified image
    idx_tmp = incorrect_idx[i]
    # Plot the image
    ax.imshow(X[idx_tmp, 0, :, :], cmap=cm.Greys)
    ax.set(xticks=[], yticks=[], title='True: {:}; Pred. {:}'.format(y[idx_tmp], y_hat[idx_tmp]))